In [1]:
%load_ext autoreload
%autoreload 2
import re
import pandas as pd

from courtscraper.data_utils.consts import DOCCS_FOIL_TXT_PATH
from courtscraper.data_utils.consts import DOCCS_FOIL_TXT_PATH, DOCCS_FOIL_XLSX, TRACKER_PATH, \
    IGNORE, ETHNICITIES, CRIMES, COUNTIES, GENERATED_MATCHES
from courtscraper.ny.doccs_foil_text_to_xlsx import gen_xlsx

In [2]:
# doccs_df = pd.read_excel(DOCCS_FOIL_XLSX)
for line in open(DOCCS_FOIL_TXT_PATH, 'r'):
    if 'LIFE' not in line:
        print(line)
        
# Oldest age juvenile offender, youthful offender
# 

In [54]:
def gen_convictions_df():
    convictions = []
    crimes = ['Most Serious Crime', 'Second Crime', 'Third Crime']
    counties = ['County of Indictment 1', 'County of Indictment 2', 'County of Indictment 3']
    for idx, fieldname in enumerate(crimes):
        for _, line in doccs_df.iterrows():
            if line[fieldname] == 'MURDER 2ND':
                convictions.append([
                    line['DIN'],
                    line['Name'],
                    line['Date of Birth'],
                    line['Ethnicity'],
                    line[fieldname],
                    line[counties[idx]],
                    line['Min Prison Term in Months']
                ])
    convs = pd.DataFrame(convictions)
    convs.columns = ['DIN', 'Name', 'DOB', 'Race', 'Crime', 'County', 'Min Term (Months)']

    convs.Name = convs.Name.apply(lambda x: x.title())
    convs.Race = convs.Race.apply(lambda x: x.title())
    convs.County = convs.County.apply(lambda x: x.title())
    convs.County = convs.County.apply(lambda x: 'St. Lawrence' if x == 'St Lawrence' else x)
    convs['YOB'] = convs.DOB.apply(lambda x: int(str(x)[:4]))
    convs.Race = convs.Race.apply(lambda x: x if x in ['Black', 'White', 'Hispanic', 'Asian'] else 'Other-Unknown')
    convs = convs.drop_duplicates()
    return convs

convs = gen_convictions_df()

In [86]:
# from collections import Counter
# Counter(convs['DOB'])

# # gen_xlsx('15A0634')
# # convs[convs['County'].apply(lambda x: type(x) != str)]
# doccs_df

/Users/eli/projects/felony-murder/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [116]:
tracker = pd.read_excel(TRACKER_PATH, sheet_name=1)

name_matches = {}
with open(GENERATED_MATCHES, 'w', encoding='utf-8') as file:
    for _, row in tracker.iterrows():
        if isinstance(row['Name'], str):
            continue
        if row['Crime Year'] != 'Unknown' and row['Crime Year'] >= 2020:
            continue
        match_convs = convs[convs['County'].apply(lambda x: x == row['Disposition County'])]
#         print(row['Disposition County'])
#         print(match_convs)
        min_term = row['Min Prison Term in Months']
        if str(min_term) in ['nan', 'Unknown']:
            match_convs = match_convs[
                match_convs['Min Term (Months)'].apply(
                    lambda x: str(x) == 'nan')]
        else:
            match_convs = match_convs[
                match_convs['Min Term (Months)'].apply(
                    lambda x: str(x) != 'nan' and int(x) == int(min_term))]

        if match_convs.empty:
            continue
        match_convs = match_convs[
            match_convs['Race'].apply(
                lambda x: x == row['Race/Ethnicity of Arrestee'])]
        if match_convs.empty:
            continue
        if row['Crime Year'] != 'Unknown':
            potential_yobs = [(row['Crime Year'] - row['Age at Crime']),
                             (row['Crime Year'] - row['Age at Crime']) - 1]
            match_convs = match_convs[
                match_convs['YOB'].apply(
                    lambda x: x in potential_yobs)]
        if match_convs.empty:
            continue
        match_convs = match_convs[match_convs['DIN'].apply(
                lambda x: int(x[:2]) >= int(str(row['Arrest Year'])[2:])
            )]

        if not match_convs.empty:
            match_convs = match_convs.set_index('DIN')
            
            file.write(f"Original Number: {row['Original Number']} \n")
            file.write(f"Crime Year: {row['Crime Year']} Age at Arrest: {row['Age at Crime']}")
            file.write(f" Arrest Year: {row['Arrest Year']} ")
            file.write(f" Race: {row['Race/Ethnicity of Arrestee']} Disposition County: {row['Disposition County']} \n")
            file.write(str(match_convs[['Name', 'DOB', 'Race', 'County', 'Min Term (Months)']]))
            file.write('\n\n----------------------\n')
            name_matches[row['Original Number']] = ', '.join(match_convs['Name']) 
                
tracker['AutoName'] = tracker['Original Number'].apply(lambda x: name_matches.get(x, 'None Found'))
tracker.to_excel(TRACKER_PATH)
        

/Users/eli/projects/felony-murder/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
from collections import Counter
tracker = pd.read_excel(TRACKER_PATH)

names_tracker = tracker[tracker['Name'].apply(lambda x: type(x) == str)]
# tracker = tracker[tracker['Max Prison Term in Months'] == 'Life Sentence']
# tracker.to_excel('LifeSentenceNoNameTracker.xlsx')
len(tracker)

1232

Counter({2008: 29,
         2018: 24,
         2016: 23,
         2010: 22,
         2012: 17,
         2015: 17,
         2013: 17,
         2011: 16,
         2014: 16,
         2017: 13,
         2009: 13,
         2019: 12,
         2021: 6,
         2020: 2})

In [14]:
Counter(names_tracker['Top Conviction Charge'])
# names_tracker.columns

Counter({'PL 125.25 MURDER-2ND': 206,
         'PL 160.15 ROBBERY-1ST': 5,
         'PL 125.20 MANSLAUGHTER -1ST': 5,
         'PL 125.27 MURDER-1ST': 5,
         'PL 140.30 BURGLARY-1ST': 3,
         nan: 1,
         'PL 150.20 ARSON-1ST': 1,
         'PL 115.00 CRIMINAL FACILITATION-4TH': 1})

In [16]:
# tracker = pd.read_excel(TRACKER_PATH)
no_name_tracker = tracker[tracker['Name'].apply(lambda x: type(x) != str)]

Counter(no_name_tracker['Top Conviction Charge'])
{'PL 125.25 MURDER-2ND': 131}
    


{'PL 125.25 MURDER-2ND': 131}

In [36]:
fm_top = no_name_tracker[no_name_tracker['Top Conviction Charge'] == 'PL 125.25 MURDER-2ND']
# start in 2008 - 2019
Counter(fm_top['Disposition Year'])
# len(fm_top)
Counter(fm_top['Max Prison Term in Months'])
fm_top[fm_top['Max Prison Term in Months'] != 'Life Sentence']

# for k, v in fm_top[['Top Conviction Charge', 'Max Prison Term in Months']].iterrows():
    

,Unnamed: 0,Original Number,Crime Year,Arrest Year,Arrest County,Age at Crime,Sex of Arrestee,Race/Ethnicity of Arrestee,Disposition County,Top Arrest Charge,...,Jail Term in Days,Min Prison Term in Months,Max Prison Term in Months,Lucy Match,Basis of Charge,Eli Match,Name,Source,Notes,AutoName
6,6,44,2006,2006,Kings,21,Male,White,Kings,PL 125.25 MURDER-2ND,...,NaN,NaN,240,0 matches,NaN,0 potential matches,NaN,NaN,NaN,None Found
14,14,692,2008,2009,New York,19,Male,Hispanic,New York,PL 125.25 MURDER-2ND,...,NaN,NaN,120,0 matches,NaN,0 potential matches,NaN,NaN,NaN,None Found
15,15,714,2011,2012,New York,16,Female,Black,New York,PL 125.25 MURDER-2ND,...,NaN,16,48,0 matches,NaN,0 potential matches,NaN,NaN,NaN,None Found
22,22,476,2011,2011,Suffolk,23,Male,Hispanic,Suffolk,PL 125.25 MURDER-2ND,...,NaN,NaN,252,0 matches,NaN,0 potential matches,NaN,NaN,NaN,None Found
23,23,696,2009,2010,Suffolk,21,Male,Black,Suffolk,PL 125.25 MURDER-2ND,...,NaN,NaN,120,0 matches,NaN,0 potential matches,NaN,NaN,NaN,None Found
343,343,496,2008,2008,Bronx,20,Male,Black,Bronx,PL 125.25 MURDER-2ND,...,NaN,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,None Found
358,358,596,2008,2012,Bronx,24,Male,Hispanic,Bronx,PL 125.25 MURDER-2ND,...,NaN,NaN,156,NaN,NaN,NaN,NaN,NaN,NaN,None Found
383,383,745,2013,2014,Bronx,18,Male,Black,Bronx,PL 125.25 MURDER-2ND,...,NaN,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,None Found
435,435,987,2003,2007,Bronx,18,Female,Black,Bronx,PL 125.25 MURDER-2ND,...,NaN,NaN,300,NaN,NaN,NaN,NaN,NaN,NaN,None Found
451,451,1070,2007,2008,Bronx,29,Male,Black,Bronx,PL 125.25 MURDER-2ND,...,NaN,NaN,180,NaN,NaN,NaN,NaN,NaN,NaN,None Found


In [102]:
# Counter(tracker)
tracker = tracker[tracker['Name'].apply(lambda x: type(x) != str)]
Counter(tracker['Max Prison Term in Months'])
tracker = tracker[tracker['Max Prison Term in Months'] == 'Life Sentence']

In [105]:
tracker
for _, row in tracker.iterrows():
    print(row['Notes'])

Disposition county and county of commitment (incarcerated lookup) don't match: Davon K. Cabrera=Nassau, David Sanchez=Broome; couldn't match elsewhere
Disposition county and county of commitment (incarcerated lookup) don't match: Julio Moronta=New York; also appellate article cites 125.25(1); might be related to Nadine Panton and Sparkle Daniel?
Disposition county and county of commitment (incarcerated lookup) don't match: Christian Estevez=Kings; couldn't match elsewhere
Closest match is Robert Loucks but his crime was committed in 2010: https://www.the-leader.com/story/news/2012/11/16/man-gets-25-to-life/44438654007/
Disposition county and county of commitment (incarcerated lookup) don't match for Anthony Negrin=Kings County; dates don't match for crime year for Kevin J. Davis II; couldn't match either elsewhere
Disposition county and county of commitment (incarcerated lookup) don't match: Melanie T. Liverpool=New York; couldn't match elsewhere
Not Ricky Proctor = 125.25(1) (https://